# Implémentation pipeline RAG

1. Etape 1. Initialiser le modèle LLM
2. Etape 2. Créer le prompt template
3. Etape 3. Ingestion PDF
4. Etape 4. Chunks
5. Etape 5. Embedding, vector store et retriever
6. Etape 6. Chain
7. Etape 7. Query

Le choix des classes et méthodes a été réalisé à l'issue d'une réflexion autour de notre besoin métier et à l'issue d'une phase de pré-test. Nous avons choisi les modules nous paraissant les plus pertinents quant au ratio coût-pertinence.

# 1. Etape 1. Extraction du texte

La classe PyMuPDFLoader en association avec la méthode load() charge le fichier dont elle extrait le texte brut qu'elle stocke dans une variable objet constituée d'une collection de documents (à raison d'un document par page, avec les métadonnées). Parmi les nombreux extracteurs de textes assumés par Langchain, PyMuPDFLoader est de loin le plus rapide.

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_path = "plu_0.pdf"
docs = PyMuPDFLoader(pdf_path).load()

print(len(docs))
print(type(docs))
print(type(docs[0]))

172
<class 'list'>
<class 'langchain_core.documents.base.Document'>


La longueur de la variable docs est bien 172, soit le nombre de pages de notre PDF. C'est une liste d'objets langchain.

# 2. Etape 2. Chunks

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.utils import filter_complex_metadata

# Création des chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

# Ajout d'un filtre pour nettoyer les métadonnées - réduit la complexité
chunks = filter_complex_metadata(chunks)

print(len(chunks))
print(type(chunks))
print(type(chunks[0]))

594
<class 'list'>
<class 'langchain_core.documents.base.Document'>


Nous avons réduit la taille de notre objet à 594 chunks. La variable est également une liste d'objets langchain.

# 3. Etape 3. Embedding, vector store et retriever

Il s'agit à présent de créer la base de données vectorielle et le récupérateur qui va retrouver les chunks les plus pertinents. Pour cela, nous utiliserons le modèle pré-entraîné FAISS (implémenté par Meta), qui va stocker les vecteurs et les indexer. Nous paramétrerons ensuite l'objet issu de ce processus à l'aide de la méthode as_retriever() pour que celui-ci effectue un calcul de similarité sémantique avec la requête (par défaut : similarité cosinus), en fonction des k-voisins que nous lui indiquerons et ce à partir d'un seuil d'acceptabilité ("score_threshold"). Pour cela, nous utiliserons la méthode as_retriever(). Le nombre de k-voisins à sélectionner est également déterminant, il s'agira de trouver le k qui dans notre cas d'usage augmente les chances de contenir le bon document, tout en réduisant les risques de noyer l'information dans des documents non pertinents. Nous choisissons dans notre implémentation de renvoyer un seul document, en misant sur le fait que le retriever choisira d'emblée le bon. Nous ferons varier ces éléments dans la partie expérimentation. Nous choisissons la méthode de recherche par calcul du score de similarité cosinus idéale dans notre cas car nous recherchons des résultats très pertinents par rapport à notre requête (nous ne demandons surtout pas de la variabilité comme cela pourrait être calculé à partir de l'approche "mmr" ou Maximal Marginal Relevance).

In [3]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = FastEmbedEmbeddings() # par défaut, model_name de la classe: "BAAI/bge-small-en-v1.5"

vector_store = FAISS.from_documents(documents=chunks, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold": 0.7,},
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Nous pouvons à présent observer le fonctionnement du retriever, grâce à la méthode invoke() qui déclenche le processus de récupération, si on lui donne directement la requête comme une chaîne de caractères.

In [4]:
retrieved_chunks = retriever.invoke("Quelle est la hauteur maximum des constructions dans la zone U1 ?")
retrieved_chunks

[Document(metadata={'source': 'plu_0.pdf', 'file_path': 'plu_0.pdf', 'page': 8, 'total_pages': 172, 'format': 'PDF 1.4', 'title': 'GIG_PLU_Modif1_APPRO_4a_Reglt_20210628', 'author': 'Eloïse DE CARVALHO', 'subject': '', 'keywords': '', 'creator': 'Word', 'producer': 'Mac OS X 10.12.6 Quartz PDFContext', 'creationDate': "D:20210628092938Z00'00'", 'modDate': "D:20210628092938Z00'00'", 'trapped': ''}, page_content='la construction ou de l’installation, cheminées, antennes et autres ouvrages techniques exclus. \n2) Hauteur maximum au faîtage \nToute construction ou installation ne peut excéder 12,50 mètres de «hauteur  maximum». \nEn cas d’extension ou de rénovation de bâtiments existant ayant une hauteur supérieure au maximum \nindiqué ci dessus, la hauteur pourra atteindre celle de la construction existante. \n3) Hauteur relative \nPar rapport à la voie, la hauteur de toute constructions doit être telle que la différence d’altitude entre \ntout point du bâtiment et tout point de l’alignem

Nous observons que le chunk retrouvé correspond bien à celui qui contient l'information désirée, à savoir 12.50 mètres.

# 4. Etape 4. Initialisation du LLM

Nous utilisons un LLM fourni par la plateforme Ollama qui permet de charger divers modèles de langage dans un environnement local, que Langchain prend ensuite en charge. Ici nous sélectionnons Llama2, mais nous testerons différents modèles dans notre partie expérimentation. Il est important de présenter les paramètres qui vont contrôler le comportement du modèle.
- temperature : si elle est proche de 0, le modèle reste déterministe, produisant toujours les réponses les plus probables, admettant peu de diversité ou de créativité. En revanche, si elle est plus élevée (1, 2 ou 3), la température rend le modèle plus créatif, au détriment parfois de la cohérence. Dans notre cas, nous avons impérativement besoin de réponses précises, sans hallucinations, et reproductibles. Nous choisissons donc d'emblée une température de 0, et nous ne ferons plus varier ce paramètre ensuite.
- top_k : détermine combien d'options (k) parmi les plus probables le LLM choisira lors de la génération de chaque token. Ici aussi nous choississons une valeur faible car nous voulons des chiffres et de la précision.
- top_p : sélectionne les tokens suivants selon une distribution de probabilité. Nous utilisons une valeur de 1 (soit 100%) pour prendre en compte toutes les possibilités de réponses au moment de la génération. Associé à la température de 0 et au top_k de 1, ce paramètre nous semble favorable pour maximiser les réponses précises et pertinentes.

In [5]:
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="llama2", temperature=0, top_k=1, top_p=1)

# 5. Etape 5. Création du prompt template

Le prompt permet de guider le LLM dans sa manière d'interagir avec le contexte et de structurer sa réponse. Encore une fois, il nous faut priviléger un prompt à l'image de ce que nous attendons de la génération : de la concision et de la précision.
Pour une documentation sur les bonnes pratiques en matière de formulation de prompt (hors RAG), voir https://medium.com/the-modern-scientist/best-prompt-techniques-for-best-llm-responses-24d2ff4f6bca et 
https://www.promptingguide.ai/introduction/examples#information-extraction

Il a été montré qu'une invite simple et précise augmente la qualité de la réponse https://arxiv.org/pdf/2312.16171.

In [6]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    <s> [INST] Vous êtes un assistant chargé de l'analyse des documents d'urbanisme. 
    Votre tâche est d'extraire des chiffres à partir du contexte.
    Donnez uniquement le chiffre, rien d'autre. 
    [/INST] </s> 
    [INST] Question: {question} 
    Context: {context} 
    Answer: [/INST]
    """
)

# 6. Etape 6. Chain

Cette étape concerne la création de la chaîne de traitement : langchain permet grâce à une syntaxe spécifique d'assembler toutes les étapes du pipeline en une seule commande de traitement. Nous retrouvons les 2 points clés : récupérer la requête et le contexte pertinent à partir du vector store, puis générer une réponse par le LLM. La classe RunnablePassthrough permet de passer la question telle quelle dans la chaîne et StrOutputParser permet de parser la sortie du modèle en transformant la réponse générée en une chaîne de caractère.

In [31]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

chain = ({"context": retriever, "question": RunnablePassthrough()}
         | prompt_template
         | model
         | StrOutputParser()
         )

# 7. Etape 7. Query

In [32]:
question = "Quelle est la hauteur maximum des constructions dans la zone U1 ?"
answer = chain.invoke(question)
print(answer)


12.5


# Classe complète et utilisation

Ajout d'un print() pour le retriever permettant de visualiser le chunk choisi, et du module time pour chronométrer les temps de l'ingestion, du retrieval et de la réponse du LLM.

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.utils import filter_complex_metadata
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
import time

In [2]:
class RagPLU:
    vector_store = None
    retriever = None
    chain = None

    def __init__(self, model_name="mistral", chunk_size=4084, chunk_overlap=0):
        self.model = ChatOllama(model=model_name, temperature=0, top_k=1, top_p=1)
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        self.prompt = PromptTemplate.from_template(
            """
            <s> [INST] Vous êtes un assistant chargé de l'analyse des documents d'urbanisme. 
            Votre tâche est d'extraire des chiffres à partir du contexte.
            Donnez uniquement le chiffre, pas de texte. 
            [/INST] </s> 
            [INST] Question: {question} 
            Context: {context} 
            Answer: [/INST]
            """
        )

    def ingest(self, pdf_path: str):
        start_time = time.time()
        docs = PyMuPDFLoader(file_path=pdf_path).load()
        chunks = self.text_splitter.split_documents(docs)
        chunks = filter_complex_metadata(chunks)

        vector_store = FAISS.from_documents(documents=chunks, embedding=FastEmbedEmbeddings())
        self.retriever = vector_store.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={
                "k": 1,
                "score_threshold": 0.6,
            },
        )

        self.chain = ({"context": self.retriever, "question": RunnablePassthrough()}
                      | self.prompt
                      | self.model
                      | StrOutputParser())
        
        ingestion_time = time.time() - start_time 
        print(f"Ingestion completed in {ingestion_time:.2f} sec")

    def ask(self, query: str):
        if not self.chain:
            return "Please, add a PDF document first."
        
        start_time = time.time()
        retrieved_chunks = self.retriever.invoke(query)
        print(retrieved_chunks)
        retriever_time = time.time() - start_time 
        print(f"Retrieval completed in {retriever_time:.2f} sec")

        start_time = time.time()
        answer = self.chain.invoke(query)
        query_time = time.time() - start_time
        print(f"Query answered in {query_time:.2f} sec")

        return answer

    def clear(self):
        self.vector_store = None
        self.retriever = None
        self.chain = None

In [7]:
#### Utilisation de la classe ####

# 1. Instanciation de la classe
pdf_query = RagPLU() # paramètres par défaut

# 2. Ingestion du PDF
pdf_query.ingest("plu_0.pdf")

# 3. Question
answer = pdf_query.ask("Quelle est la hauteur maximum des constructions dans la zone U1 ?")
print(answer)

# 4. Effacer les variables stockées
pdf_query.clear()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Ingestion completed in 2.28 min
[Document(metadata={'source': 'plu_0.pdf', 'file_path': 'plu_0.pdf', 'page': 8, 'total_pages': 172, 'format': 'PDF 1.4', 'title': 'GIG_PLU_Modif1_APPRO_4a_Reglt_20210628', 'author': 'Eloïse DE CARVALHO', 'subject': '', 'keywords': '', 'creator': 'Word', 'producer': 'Mac OS X 10.12.6 Quartz PDFContext', 'creationDate': "D:20210628092938Z00'00'", 'modDate': "D:20210628092938Z00'00'", 'trapped': ''}, page_content='la construction ou de l’installation, cheminées, antennes et autres ouvrages techniques exclus. \n2) Hauteur maximum au faîtage \nToute construction ou installation ne peut excéder 12,50 mètres de «hauteur  maximum». \nEn cas d’extension ou de rénovation de bâtiments existant ayant une hauteur supérieure au maximum \nindiqué ci dessus, la hauteur pourra atteindre celle de la construction existante. \n3) Hauteur relative \nPar rapport à la voie, la hauteur de toute constructions doit être telle que la différence d’altitude entre \ntout point du bât

# Boucle de questions

Il est à noter que le client fournira une liste de sous-sections, qui alimentera la liste de questions pour chaque indicateur (hauteur, emprise au sol).

In [3]:
import re
import pandas as pd

# 1. Instanciation de la classe
pdf_query = RagPLU(model_name="llama2", chunk_size=4096, chunk_overlap=0)

# 2. Ingestion du PDF
pdf_query.ingest("plu_1.pdf")

# 3. Liste de questions
questions = [
    #"Quelle est la hauteur maximum des constructions dans la zone U1 ?",
    "Quelle est la hauteur maximum des constructions dans la zone U2 ?",
    "Quelle est la hauteur maximum des constructions dans la zone 1AUeq ?",
    #"Quelle est la hauteur maximum des constructions dans la zone 1AUE ?",
    #"Quelle est l'emprise au sol dans la zone U1 ?",
    #"Quelle est l'emprise au sol dans la zone U2 ?",
    #"Quelle est l'emprise au sol dans la zone 1AUeq ?",
    #"Quelle est l'emprise au sol dans la zone 1AUE ?"
]

# 4. Liste pour stocker les réponses numériques
answers = []

# 5. Boucle pour poser chaque question
for question in questions:
    full_answer = pdf_query.ask(question)

    # Extraire le nombre trouvé dans la réponse
    number = re.findall(r"\d+\.?\d*", full_answer)
    if number:
        answers.append(number[0])
    else:
        answers.append("Non trouvé")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Ingestion completed in 129.37 sec
[Document(metadata={'source': 'plu_1.pdf', 'file_path': 'plu_1.pdf', 'page': 22, 'total_pages': 172, 'format': 'PDF 1.4', 'title': 'GIG_PLU_Modif1_APPRO_4a_Reglt_20210628', 'author': 'Eloïse DE CARVALHO', 'subject': '', 'keywords': '', 'creator': 'Word', 'producer': 'Mac OS X 10.12.6 Quartz PDFContext', 'creationDate': "D:20210628092938Z00'00'", 'modDate': "D:20210628092938Z00'00'", 'trapped': ''}, page_content='Dispositions applicables aux zones Urbaines \nVille de Gigean (34) // Plan Local d’Urbanisme // RÈGLEMENT \n1ère Modification // Document approuvé le : 29 juin 2021 // page 23 \nREGL. \nN \nA \nAU \nU \nZONE U2 \n \n> Article 10 :  Hauteur maximum des constructions \n1) Définition et mesure de la hauteur maximum des constructions \nLa « hauteur maximum » est mesurée verticalement à partir du sol naturel avant travaux en tout point de \nla construction ou de l’installation, cheminées, antennes et autres ouvrages techniques exclus. \n2) Hauteur m

In [4]:
answers

['11', '12.5']

# Insersion en base de données

Il est nécessaire de créer une instance de connexion à la base de données PostgreSQL. Ceci se réalise grâce à la librairie psycopg2.

Les réponses seront insérées directement dans la table à partir de la variable answers. Les valeurs d'idterritoire, codcom, annee sont celles de la collectivité. Celles de la zone et de la sections sont récupérées dans la liste fournie par le client.

In [ ]:
import psycopg2
from RagPLU import answers # récupération des données de la variable answers dans le fichier rag

class PLUReglementInserter:
    def __init__(self, dbname, user, password, host, port):
        self.conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        self.cursor = self.conn.cursor()

    def insert_data(self, idterritoire, codcom, annee, zone, section, answers):
        for answer in answers:
            self.cursor.execute('''
                INSERT INTO plu_reglement (idterritoire, codcom, annee, zone, section, hauteur, emprise)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            ''', (idterritoire, codcom, annee, zone, section, answer['hauteur'], answer['emprise']))
    
    def commit_and_close(self):
        self.conn.commit()
        self.cursor.close()
        self.conn.close()

In [ ]:
#### Utilisation de la classe ####
if __name__ == "__main__":

    # Données de connexion
    dbname = "xxx"
    user = "xxx"
    password = "xxx"
    host = "localhost"
    port = "5432"

    # Création d'une instance de la classe
    inserter = PLUReglementInserter(dbname, user, password, host, port)
    
    # Exemples de valeurs à insérer (récupérées dans les données clients en même temps que l'import du fichier)
    idterritoire, codcom, annee, zone, section = (2024125006, 125006, 2024, "U", "U1")
    
    # answers est récupérée dans le fichier rag
    
    # Insertion des données
    inserter.insert_data(idterritoire, codcom, annee, zone, section, answers)
    
    # Sauvegarder et fermer la connexion
    inserter.commit_and_close()